In [34]:
# Import dependancies
from bs4 import BeautifulSoup as bs
from splinter import Browser

import pymongo
import pandas as pd
import time
import requests
import os

In [35]:
# Chrome driver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [36]:
# MARS NEWS
news_url= 'https://mars.nasa.gov/news/'
browser.visit(news_url)
html=browser.html
news_soup = bs(html, 'html.parser')

In [37]:
# MARS NEWS: Title and Paragraph
news_title = news_soup.find_all('div', class_='content_title')[0].text
news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
print(news_p)

Mars Now
NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [38]:
# MARS IMAGE
image_url= 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)
html=browser.html
image_soup = bs(html, 'html.parser')

In [39]:
section=image_soup.find('div', class_ = "default floating_text_area ms-layer")
featured_image=section.find('footer')
featured_image_url = 'https://www.jpl.nasa.gov' + featured_image.find('a')['data-fancybox-href']
print(str(featured_image_url))

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19404_ip.jpg


In [40]:
# MARS FACTS
facts_url = "https://space-facts.com/mars/"
tables = pd.read_html(facts_url)
facts_df = tables[2]
facts_df.columns = ["Description", "Mars"]
facts_df.set_index('Description')
facts_df

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [41]:
html_facts=facts_df.to_html()
html_facts=html_facts.replace('\n', '')
print(html_facts)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Description</th>      <th>Mars</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomer

In [48]:
#MARS HEMISPHERES
home_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(home_url)
html=browser.html
home_soup=bs(html, 'html.parser')

In [49]:
base_url = 'https://astrogeology.usgs.gov/'
page_list = home_soup.find_all('div', class_ = 'item')
links = []
hemisphere_image_urls = []

for page in page_list: 
    href = page.find('a', class_ = 'itemLink product-item')
    link = base_url + href['href']
    links.append(link)

time.sleep(1)

for link in links:
    hemisphere_dict = {}
    browser.visit(link)
    html = browser.html
    page_soup = bs(html, 'lxml')
    title_block = page_soup.find('div', class_ = 'content')
    title = title_block.find('h2', class_ = 'title').text
    hemisphere_dict["title"] = title
    img_block = page_soup.find('div', class_ = 'downloads')
    img = img_block.find('a')['href']
    hemisphere_dict['img_url'] = img
    hemisphere_image_urls.append(hemisphere_dict)

In [50]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [52]:
#MARS DICTIONARY
mars_dict = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "fact_table": str(html_facts),
        "hemisphere_images": hemisphere_image_urls}

mars_dict

{'news_title': 'Mars Now',
 'news_p': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19404_ip.jpg',
 'fact_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Description</th>      <th>Mars</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbi